# IBM Ponder - May 2020 - Challenge

Solution by Walter Sebastian Gisler

The original problem statement can be found here: https://www.research.ibm.com/haifa/ponderthis/challenges/May2020.html

On Saturday, April 11, 2020, the mathematical genius John Horton Conway passed away.

This month's challenge is dedicated, with deep love, to his memory.

Conway researched many topics in mathematics. One of his most famous inventions is the Game of Life.
In the standard game, each cell has eight neighbors; in our version, each cell has only four neighbors (only those cells with adjacent edges).

The standard rules can be formulated as 000100000;001100000 (if the cell is empty, it is born if it has exactly three live neighbors; if the cell is alive, it stays alive if it has two or three live neighbors).

In our version of the game, the rules 01100;00010 mean that a cell is born if it has one or two neighbors, and stays alive if it has three. If we start with a single cell in the middle of an 11x11 torus board, then after 15 generations, you will have an alternating chess-like pattern, and after 16 steps, just the four corners.
Your task, this month, is to find rules for our version of the game and an initial input on an 11x11 torus board that will lead, after at least 100,000 generations, to a 72-long cycle.

## Verifying the sample

First, let's see if we can reproduce the sample. This will help us to confirm that we understand the rules of the game and we can prepare some code that we might be able to reuse later.

We will represent a torus board as a two dimensional array, for example: [[0,0,0],[0,0,0],[0,0,0]] for an empty (no alive cells) 3x3 torus board. An alive cell would hold the value 1.

*__Note:__ a torus board is an infinite board. The right side of it "touches" the left side, and the top side "touches" the bottom side. Hence, every cell has exactly 4 neighbors.*

Furthermore, let's assume the rules are interpreted as follows:

The rules are separated by a ';'. The first part gives the conditions that a cell is born. If the 3rd number is a 0, this means, that a cell is born if it has 2 (3-1) alive neighbors. Note, that we could also have rules that say that a cell becomes alive if it has no (0) alive neighbors. Similarly, the second part of the rule gives the condition that a cell stays alive. If the 3rd and 4th position are 1, this means that if a cell has 2 or 3 alive neighbor cells and the cell itself is alive, it will stay alive.

For visualization, let's define a method to draw the board:

In [1]:
def draw_board(board):
    for row in board:
        line = ''
        for col in row:
            if col == 1:
                line += 'X'
            else:
                line += 'O'
        print(line)
    print()

Next, we need a method to find the next generation based on the rules:

In [2]:
def next_board(board, rule_born, rule_stay_alive):
    new_board = [[0 for i in range(len(board))] for j in range(len(board))]
    for i in range(len(board)):
        for j in range(len(board)):
            surrounding_alive = 0
            surrounding_alive += board[i-1][j]
            if i < len(board)-1:
                surrounding_alive += board[i+1][j]
            else:
                surrounding_alive += board[0][j]
            surrounding_alive += board[i][j-1]
            if j < len(board)-1:
                surrounding_alive += board[i][j+1]
            else:
                surrounding_alive += board[i][0]
            if board[i][j] == 0:
                new_board[i][j] = 1 if rule_born[surrounding_alive] == 1 else 0
            else:
                new_board[i][j] = 1 if rule_stay_alive[surrounding_alive] == 1 else 0
    return new_board

Let's also add a method to iterate and draw each board:

In [3]:
def iterate(board, generations, rule_born, rule_stay_alive):
    for gen in range(1,generations+1):
        print('After %i generations'%gen)
        board = next_board(board, rule_born, rule_stay_alive)
        draw_board(board)

And finally, let's construct the initial board, where only the center cell is alive and iterate through 16 generations:

In [4]:
initial_board = [[0]*11 for i in range(11)]
initial_board[5][5] = 1

draw_board(initial_board)
iterate(initial_board, 16, [0,1,1,0,0], [0,0,0,1,0])

OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOXOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO

After 1 generations
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOXOOOOO
OOOOXOXOOOO
OOOOOXOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO

After 2 generations
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOXOOOOO
OOOOXOXOOOO
OOOXOOOXOOO
OOOOXOXOOOO
OOOOOXOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO

After 3 generations
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOXOOOOO
OOOOXOXOOOO
OOOXOOOXOOO
OOXOOOOOXOO
OOOXOOOXOOO
OOOOXOXOOOO
OOOOOXOOOOO
OOOOOOOOOOO
OOOOOOOOOOO

After 4 generations
OOOOOOOOOOO
OOOOOXOOOOO
OOOOXOXOOOO
OOOXOOOXOOO
OOXOXOXOXOO
OXOOOOOOOXO
OOXOXOXOXOO
OOOXOOOXOOO
OOOOXOXOOOO
OOOOOXOOOOO
OOOOOOOOOOO

After 5 generations
OOOOOXOOOOO
OOOOXOXOOOO
OOOXOOOXOOO
OOXOOOOOXOO
OXOOOXOOOXO
XOOOXOXOOOX
OXOOOXOOOXO
OOXOOOOOXOO
OOOXOOOXOOO
OOOOXOXOOOO
OOOOOXOOOOO

After 6 generations
OOOOXOXOOOO
OOOXOOOXOOO
OOXOXOXOXOO
OXOXOXOXOXO
XOXOXOXOXOX
OOOXOOOXOOO
XOXOXOXOXO

As expected, the 15th iteration is a perfectly alternating chess board, and the 16th generation has its four alive cells in the corners of the board.

## Detecting circles

A circle is present, if at some point, we will encounter the same board a second time. If we encountered the board configuration the first time in round x, and the second time in round y, the length of the circle is y-x. We can store each board in a compact and unique format in a dictionary. The key is the encoding of the board, and the value is the generation where the board was first encountered.

Let's define a method to turn a 2-dimensional board array into a unique string. We could use some smart compression methods here or just hash the board, but I believe for this purpose, just creating a string is perfectly fine. It's slow and inefficient but will do the job.

In [5]:
def board_string(board):
    result = ''
    for row in board:
        for col in row:
            result += str(col)
    return result

Next, let's implement a second version of the iterate method that will not draw the boards, but simply keep track of the boards we have encountered so far and tell us if there is a circle at some point:

In [6]:
def silent_iterate(board, generations, rule_born, rule_stay_alive):
    past_boards = dict()
    past_boards[board_string(board)] = 0
    found = False
    for gen in range(1,generations+1):
        board = next_board(board, rule_born, rule_stay_alive)
        bs = board_string(board)
        if bs in past_boards:
            found = True
            print('Cycle of length %i found after %i generations'%(gen-past_boards[bs], past_boards[bs]))
            return board # We will return the board that is the start of the cycle
        else:
            past_boards[bs] = gen
    if not found:
        print('No cycle found')

Let us now test this with a sample to find out if there are any cycles in the first 100000 iterations:

In [7]:
c = silent_iterate(initial_board, 100000, [0,1,1,0,1], [0,1,1,1,0])

Cycle of length 10 found after 36 generations


## Brute Force

I did a few small experiments using a MIP model to see whether I could find a 72-long cycle and then use a second MIP model to find a board that will lead to the start board of the cycle. Both parts turned out to be difficult. Generating a cycle that is longer than 8 or 9 using a MIP is challenging. Furthermore, I quickly realized that not every board necessarily has a predecessor, so finding a 100000 long sequence that will lead to a specific board is very hard, if not impossible. The game of life can't be predicted, and similary, it doesn't seem to be able to predict whether a board has any predecessors, or how many predecessors it has.

I therefore tried a Brute Force approach which, to my surprise, actually worked. I adjusted some of the methods to make sure they don't generate so much output. I tried generating random start solutions and random rules. I noticed that some rules more frequently generate solutions with a 72-long cycle, which is why I decided to concentrate on one set of rules (9 rules, instead of 1024). However, the code that I used with random rules, which helped me realize that only 9 rules generate 72-long cycles frequently is also included in this repository (run_randomly.py). It works too, but takes significantly more time than the code below.

In [8]:
from random import random

def silent_iterate_2(board, generations, rule_born, rule_stay_alive):
    past_boards = dict()
    past_boards[board_string(board)] = 0
    found = False
    for gen in range(1,generations+1):
        board = next_board(board, rule_born, rule_stay_alive)
        bs = board_string(board)
        if bs in past_boards:
            found = True
            print('Cycle of length %i found after %i generations'%(gen-past_boards[bs], past_boards[bs]))
            return (gen-past_boards[bs], past_boards[bs])
            break
        else:
            past_boards[bs] = gen
    if not found:
        return (0,0)
        
# generate all possible rules:
def test_rule_on_start_board(start_board, target_cycle_length):
    for rules in ['0011000101', '0011000100', '0100100101', '0110111000',
                  '0111110010', '1011000001', '0100100101', '0011000001', '0011000101']: # this are the rules that we found to produce 72-long cycles frequently. We could just use all rules instead, but it would take much more time
        while len(rules) < 10:
            rules = '0'+rules
        rule_born = [int(rules[0]), int(rules[1]), int(rules[2]), int(rules[3]), int(rules[4])]
        rule_alive = [int(rules[5]), int(rules[6]), int(rules[7]), int(rules[8]), int(rules[9])]
        result = silent_iterate_2(start_board, 150000, rule_born, rule_alive)
        if result[0] == target_cycle_length:
            print('Did it.')
            print(rule_born)
            print(rule_alive)
            print(start_board)
            return result
    return (0,0)

from time import time
running = True
start_time = time()
while running and time()-start_time < 50:
    sb = [[0]*11 for i in range(11)]
    randomfactor = random()
    for i in range(11):
        for j in range(11):
            if random() > randomfactor:
                sb[i][j] = 1
    result = test_rule_on_start_board(sb, 72)
    if result[0] == 72 and result[1] > 100000:
        running = False
        print('Time used: '+str(time()-start_time))

Cycle of length 1 found after 1 generations
Cycle of length 1 found after 1 generations
Cycle of length 4 found after 219 generations
Cycle of length 24 found after 119 generations
Cycle of length 24 found after 119 generations
Cycle of length 1 found after 1 generations
Cycle of length 1 found after 1 generations


As you can see, I set a time limit for this, because it would simply take too long to run this in a Jupyter notebook. If we would keep it running, we would see, that this set of rules is generating 72-long cycles frequently and after some time it will eventually come up with a 72-long cycle after more than 100000 generations. 

In reality I used the file "run.py" which is included in this repository. I was running this with 80 processes at a time and had 2 solutions within 10 minutes. If you can not run this with so many processes, you might have to wait for a very long to find a solution. However, there is plenty of potential to improve this code. A C or Cython implementation would probably bring a 200X increase in speed. Also, the way I detect cycles and store values in a hash dictionary is far from ideal, performance wise. So, even if you don't have a massive machine at your disposal, you could still find a solution to this problem within a reasonable amount of time on a simple laptop.

Below is one solution I found, which we will now verify.

In [9]:
rule_born = [0, 1, 0, 0, 1]
rule_alive = [0, 0, 1, 0, 1]
winner_board = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1], 
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [10]:
start_board = silent_iterate(winner_board, 200000, rule_born, rule_alive)

Cycle of length 72 found after 121224 generations


The silent_iterate method returns the first board of the cycle. We can now use the iterate method to print out the whole cycle.

In [11]:
print(start_board)

iterate(start_board, 73, rule_born, rule_alive)

[[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]]
After 1 generations
OXOOOXOOXXO
OXXOXXOOOOO
OXXOXXOOOOO
XOOOOOXXXXX
XOOOOOXXOOX
OOOOOOOOOOO
XOOOOOXXOOX
XOOOOOXXXXX
OXXOXXOOOOO
OXXOXXOOOOO
OXOOOXOOXXO

After 2 generations
XXOOOXXXXXX
XOXOXOXOXXO
OXXOXXOXXXX
XOXOXOXOXXO
XXOOOXXXOOX
OOOOOOOOOOO
XXOOOXXXOOX
XOXOXOXOXXO
OXXOXXOXXXX
XOXOXOXOXXO
XXOOOXXXXXX

After 3 generations
XXOOOXXOXXO
OXOOOXOXOOX
XOOOOOXOXXO
OXOOOXOXXXX
OOOOOOOOOOO
OOOOOOOOOOO
OOOOOOOOOOO
OXOOOXOXXXX
XOOOOOXOXXO
OXOOOXOXOOX
XXOOOXXOXXO

After 4 generations
XOXOXOXOXXO
XOXOXOXOOOO
OOOOOOOXXXX
OOXOXOOOOOO
OXOOOXOXXXX
OOOOOOOOOOO
OXOOOXOXXXX
OOXOXOOOOOO
OOOOOOOXXXX
XOXOXOXOOOO
XOXOXOXOXXO

Af